In [ ]:
! pip install -q langchain==0.0.350
! pip install -q gradio==4.9.0
! pip install -q openai==1.3.8
! pip install -q python-dotenv==1.0.0

In [ ]:
from dotenv import load_dotenv, find_dotenv

# find .env automatically by walking up directories until it's found 
# .env is a private file that is not checked into git
# you can create one on your local folder and add your OPEN API key there like this:
# OPENAI_API_KEY=YOUR_OPENAI_API_KEY
_ = load_dotenv(find_dotenv())

In [ ]:
import gradio as gr
import openai


def predict(message, history):
    """
    Callable function for Gradio interface. This function will be called when user click send button. 
    This function call the OpenAI API to get the response.
    """
    history_openai_format = [{"role": "system", "content": "Assistant is the subject matter expert of all the Star Wars movies. Assistant able to help user to learn or explain everything related to the Star Wars movies."}]
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model='gpt-3.5-turbo',
        messages= history_openai_format,
        temperature=1.0,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        # chunk is a ChatCompletionChunk object that store the streaming response in content within
        # delta object nested within choices object
        if chunk.choices[0].delta.content is not None and len(chunk.choices[0].delta.content) != 0:
            partial_message = partial_message + chunk.choices[0].delta.content
            yield partial_message

chat_interface = gr.ChatInterface(
    predict,
    clear_btn=None,
    undo_btn=None,
    retry_btn=None,
    submit_btn="Send"
    )
chat_interface.queue().launch()

In [ ]:
# close the chat interface server
chat_interface.close()